## import modules

In [1]:
import pandas as pd

## read datafiles
- C-18 for language population
- C-13 for particular age-range population from a state

In [2]:
c18=pd.read_excel('datasets/C-18.xlsx',skiprows=6,header=None,engine='openpyxl')

In [3]:
# for newer formates like xls don't need to use `openpyxl` engine
c13=pd.read_excel('datasets/C-13.xls',skiprows=7,header=None)

### particular age groups are
- 5-9
- 10-14
- 15-19
- 20-24
- 25-29
- 30-49
- 50-69
- 70+
- Age not stated

## obtain useful data from C-13 and C-18 for age-groups
- first get particular state names for identifying specific states
- get particular age-groups from C-18 file
- make list of particular age group row/col for a particular states
- now just simply iterate through each state to get relevent data and store it into a csv file
    - to get total pop of particular age-range I have used C-13 file
    - to get total pop that speaks more than 3 languages from a state in a particular age-range I have used C-18 file 

In [4]:
# STATE_NAMES=[list(np.unique(c18.iloc[:,2].values))]
STATE_NAMES=[]
for state in c18.iloc[:,2].values:
    if not (state in STATE_NAMES):
        STATE_NAMES.append(state)
AGE_GROUPS=list(c18.iloc[1:10,4].values)
# although it is a bit of manual work but it is worth the efforts
AGE_GROUP_RANGES=[list(range(5,10)),list(range(10,15)),list(range(15,20)),list(range(20,25)),list(range(25,30)),list(range(30,50)),list(range(50,70)),list(range(70,100))+['100+'],['Age not stated']]

In [5]:
useful_data=[]
for i,state in enumerate(STATE_NAMES):
    for j,age_grp in enumerate(AGE_GROUPS):
        
        # this list is to get only the years in the particular age-group

        true_false_list=[]
        for single_year_age in c13.iloc[:,4].values:
            if single_year_age in AGE_GROUP_RANGES[j]:
                true_false_list.append(True)
            else:
                true_false_list.append(False)

        # here i is the state code
        total_pop=c13[(c13.loc[:,1]==i) & (true_false_list)].iloc[:,5].values.sum()
        
        # tri
        tri_total=c18[(c18.iloc[:,0]==i) & (c18.iloc[:,4]==age_grp) & (c18.iloc[:,3]=='Total')].iloc[0,8]

        item={
            'state-code':i,
            'state-name':state,
            'age-group':age_grp,
            'age-group-pop':total_pop,
            'tri-ratio':tri_total/total_pop,
        }

        useful_data.append(item)

In [6]:
df=pd.DataFrame(useful_data)

## age-analysis 
- get highest ratio age-group for a state and store it into csv file

In [7]:
tri_list=[]
for i in range(36):
    values=df[df['state-code']==i].sort_values(by='tri-ratio',ascending=False).iloc[0,[2,4]].values
    tri_item={
        'state/ut':i,
        'age-group':values[0],
        'percentage':round(values[1]*100,2)
    }

    tri_list.append(tri_item)

In [8]:
tri_df=pd.DataFrame(tri_list)

In [9]:
tri_df.to_csv('outputs/age-india.csv',index=False)

In [10]:
tri_df

,state/ut,age-group,percentage
0,0,20-24,11.52
1,1,20-24,29.58
2,2,20-24,8.21
3,3,15-19,45.76
4,4,50-69,42.77
5,5,20-24,3.09
6,6,15-19,7.32
7,7,70+,15.00
8,8,15-19,2.88
9,9,20-24,2.44


## observations

- in most states we have younger population(like 15-19 or 20-24) that speaks 3+ languages in highest proportion
- in some states(though very few) older population(like 50-69,70+ etc) that speaks 3+ languages in highest proportion
- in delhi we have 70+ intellectuals as the dominating and around 15% of them speak more than 3 langs